In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np
import pandas as pd
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

def make_deltas(masks):
    deltas = []
    for h in range(len(masks)):
        if h == 0:
            deltas.append([1 for _ in range(masks.shape[1])])
        else:
            deltas.append([1 for _ in range(masks.shape[1])] + (1-masks[h]) * deltas[-1])
    
    return list(deltas)


class MyDataset(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

class MyDataset2(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

def missing_data_rbf(df,rbf, batch_size, seq_len):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0)
    
    mydata  = MyDataset(dataset, seq_len)
    data = DataLoader(mydata, batch_size, shuffle=False)

    return data


def val_missing_data_rbf(df, real, rbf, batch_size, seq_len):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    real_masks = ~np.isnan(real.values)
    
    masks = masks.reshape(shp)
    real_masks = real_masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    real = torch.nan_to_num(torch.from_numpy(real.values).to(torch.float32))

    masks = torch.from_numpy(masks).to(torch.float32)
    real_masks = torch.from_numpy(real_masks).to(torch.float32)
    
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)


    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0), real.unsqueeze_(0), real_masks.unsqueeze_(0)], dim = 0)
    mydata  = MyDataset2(dataset, seq_len)
    data = DataLoader(mydata, batch_size, shuffle=False)
    
    return data

def eval_model(model, rbf, realpath, dfpath, batch_size, seq_len):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)

    dataset = val_missing_data_rbf(df,real, rbf, batch_size, seq_len)

    model.eval()
    c_list = []
    train_list = []
    real_list = []
    real_masks_list = []
    masks_list = []

    batch_x_loss = 0.0
    # batch_loss = 0.0
    # batch_d_loss = 0.0

    for data in dataset:
        data = data.to(device)
        value = data[:,0,:].clone().detach()
        real_value = data[:,4,:].clone().detach()
        real_masks = data[:,5,:].clone().detach()
        train_masks = data[:,2,:].clone().detach()

        train_list.append(value)
        real_list.append(real_value)
        masks_list.append(train_masks)
        real_masks_list.append(real_masks)

        c_hat_list, imputations, x_loss = model(data)
        c_list.append(c_hat_list)
        batch_x_loss += x_loss
        # batch_d_loss += D_loss
        # batch_loss = loss

    c_list = torch.cat(c_list, dim = 0)
    c_list_ori = c_list * torch.tensor(df.std().values, device= device).reshape(1,1,36) + torch.tensor(df.mean().values, device= device).reshape(1,1,36)

    real_list = torch.cat(real_list, dim = 0) # none scale
    real_masks_list = torch.cat(real_masks_list, dim = 0)

    masks_list = torch.cat(masks_list, dim = 0)
    train_list = torch.cat(train_list, dim = 0)

    missing_masks = real_masks_list - masks_list
    # print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    # print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(missing_masks))
    print("Original MRE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(torch.abs(real_list * missing_masks)))
    print()
    print('Train MAE :', torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(masks_list))
    print("Train MRE :", torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(torch.abs(real_list * masks_list)))
    print('--------------------------------------------------------------')


    print("x_loss: {}".format(batch_x_loss))

def val_missing_data_rbf2(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model2(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf2(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    c_hat_list, imputations, x_los = model(dataset)

    Nonscale_imputataion = pd.DataFrame(c_hat_list[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('Train MAE :', np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))
    print("Train MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(np.abs(real.values * masks.cpu().numpy())))

    return Nonscale_imputataion

In [21]:
def eval_model_ver2(model, rbf, realpath, dfpath, batch_size, seq_len):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)

    dataset = val_missing_data_rbf(df,real, rbf, batch_size, seq_len)

    model.eval()
    c_list = []
    train_list = []
    real_list = []
    real_masks_list = []
    masks_list = []

    batch_x_loss = 0.0
    # batch_loss = 0.0
    # batch_d_loss = 0.0

    for data in dataset:
        data = data.to(device)
        value = data[:,0,:].clone().detach()
        real_value = data[:,4,:].clone().detach()
        real_masks = data[:,5,:].clone().detach()
        train_masks = data[:,2,:].clone().detach()

        train_list.append(value)
        real_list.append(real_value)
        masks_list.append(train_masks)
        real_masks_list.append(real_masks)

        c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_losss = model(data)
        c_list.append(c_hat_list)
        batch_x_loss += x_loss
        # batch_d_loss += D_loss
        # batch_loss = loss

    c_list = torch.cat(c_list, dim = 0)
    c_list_ori = c_list * torch.tensor(df.std().values, device= device).reshape(1,1,36) + torch.tensor(df.mean().values, device= device).reshape(1,1,36)

    real_list = torch.cat(real_list, dim = 0) # none scale
    real_masks_list = torch.cat(real_masks_list, dim = 0)

    masks_list = torch.cat(masks_list, dim = 0)
    train_list = torch.cat(train_list, dim = 0)

    missing_masks = real_masks_list - masks_list
    # print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    # print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(missing_masks))
    print("Original MRE :", torch.sum(torch.abs((c_list_ori - real_list) * missing_masks)) / torch.sum(torch.abs(real_list * missing_masks)))
    print()
    print('Train MAE :', torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(masks_list))
    print("Train MRE :", torch.sum(torch.abs((c_list_ori - real_list) * masks_list)) / torch.sum(torch.abs(real_list * masks_list)))
    print('--------------------------------------------------------------')


    print("x_loss: {}".format(batch_x_loss))

def val_missing_data_rbf2_ver2(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model2_ver2(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf2_ver2(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_losss = model(dataset)

    Nonscale_imputataion = pd.DataFrame(c_hat_list[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(c_hat_list[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('Train MAE :', np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))
    print("Train MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * masks.cpu().numpy())) / np.sum(np.abs(real.values * masks.cpu().numpy())))

    return Nonscale_imputataion

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size).cuda() - torch.eye(input_size, input_size).cuda()
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size)).cuda()
        self.b = Parameter(torch.Tensor(output_size)).cuda()
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size).cuda()
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

In [17]:
class MGRU4_ver2(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU4_ver2, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.r_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_layer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        x_hat_loss = 0.0
        concat_loss = 0.0
        z_hat_loss = 0.0 
        c_hat_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            
            hid = hid * gamma_h
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_hat_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_c = m * v + (1 - m) * x_hat


            RG = torch.cat([x_hat, r], dim = 1)
            concat_hat = self.concat_layer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            concat_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            a_c = m * v + (1 - m) * concat_hat

            z_hat = self.z_layer(a_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)
            z_hat_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            z_c = m * v + (1 - m) * concat_hat

            '''
            z_hat = self.z_layer(a_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)
            z_hat_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)
            c_hat_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat
            '''
            gru_input = torch.cat([z_c, m], dim = 1)
            imputations.append(z_c.unsqueeze(dim = 1))
            c_hat_list.append(concat_hat.unsqueeze(1))
            
            # GRU cell
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss

def train_MGRU_ver2(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    c_hat_list2 = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        batch_x_hat_loss = 0.0
        batch_concat_loss = 0.0
        batch_z_hat_loss = 0.0
        batch_c_hat_loss = 0.0
        for data in dataset:
            data = data.to(device)
            c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss = model(data)
        
            optimizer.zero_grad()
            x_loss.backward()
            optimizer.step()
            
            batch_loss += x_loss
            batch_x_hat_loss += x_hat_loss
            batch_concat_loss += concat_loss
            batch_z_hat_loss += z_hat_loss
            #batch_c_hat_loss += c_hat_loss

        progress.set_description("loss: {},x_hat : {}, concat_hat : {}, z_hat : {}".format(batch_loss, batch_x_hat_loss, batch_concat_loss, batch_z_hat_loss))

    return x_loss
    

In [6]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = "air_20_8.0_scale.csv"
batch_size = 64
dataset3 = missing_data_rbf(df, rbfpath, batch_size, 36)

In [8]:
model_ver1_ver2 = MGRU4_ver2(36, 64)
model_ver1_ver2.to(device)
loss_list = train_MGRU_ver2(model_ver1_ver2, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver1_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver1_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 22.80183982849121,x_hat : 20.63165283203125, concat_hat : 2.1701860427856445: 100%|██████████| 1000/1000 [07:40<00:00,  2.17it/s] 


ValueError: not enough values to unpack (expected 7, got 5)

In [11]:
Nonscale_imputataion2 = eval_model2_ver2(model_ver1_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

Scale MAE : tensor(0.1830)
Scale MRE : tensor(0.2642)
Original MAE : 14.707690485121276
Original MRE : 0.20659664988297732
Train MAE : 0.9760284087207678
Train MRE : 0.011513207382233347


In [16]:
# x_hat -> z_hat -> concat

model_ver1_ver3 = MGRU4_ver2(36, 64)
model_ver1_ver3.to(device)
loss_list = train_MGRU_ver2(model_ver1_ver3, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver1_ver3, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver1_ver3, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 50.859535217285156,x_hat : 19.981374740600586, concat_hat : 7.802461624145508, z_hat : 23.075695037841797: 100%|██████████| 1000/1000 [09:39<00:00,  1.73it/s]


Original MAE : tensor(17.2612, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2426, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(4.1523, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.0490, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 50.545284271240234
------------------------------------------------------
Scale MAE : tensor(0.2055)
Scale MRE : tensor(0.2967)
Original MAE : 16.580725033398892
Original MRE : 0.23290687603171858
Train MAE : 4.133024388876397
Train MRE : 0.048753055218268365


In [18]:
# x_hat ->  concat -> z_hat

model_ver1_ver2 = MGRU4_ver2(36, 64)
model_ver1_ver2.to(device)
loss_list = train_MGRU_ver2(model_ver1_ver2, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver1_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver1_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 56.05644226074219,x_hat : 19.956138610839844, concat_hat : 13.59693717956543, z_hat : 22.503366470336914: 100%|██████████| 1000/1000 [09:27<00:00,  1.76it/s] 


Original MAE : tensor(18.9561, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2665, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(7.2893, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.0860, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 55.74321746826172
------------------------------------------------------
Scale MAE : tensor(0.2336)
Scale MRE : tensor(0.3372)
Original MAE : 18.911271203212273
Original MRE : 0.2656436970552593
Train MAE : 7.17235836960806
Train MRE : 0.08460496496943572


In [19]:
class MGRU4_ver3(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU4_ver3, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        x_hat_loss = 0.0
        concat_loss = 0.0
        z_hat_loss = 0.0 
        c_hat_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_hat_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_c = m * v + (1 - m) * x_hat

            RG = torch.cat([x_c, r], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            concat_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            a_c = m * v + (1 - m) * concat_hat

            z_hat = self.z_layer(a_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)
            z_hat_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)
            c_hat_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid = hid * gamma_h
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_loss

def train_MGRU_ver2(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    c_hat_list2 = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        batch_x_hat_loss = 0.0
        batch_concat_loss = 0.0
        batch_z_hat_loss = 0.0
        batch_c_hat_loss = 0.0
        for data in dataset:
            data = data.to(device)
            c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_loss  = model(data)
        
            optimizer.zero_grad()
            x_loss.backward()
            optimizer.step()
            
            batch_loss += x_loss
            batch_x_hat_loss += x_hat_loss
            batch_concat_loss += concat_loss
            batch_z_hat_loss += z_hat_loss
            batch_c_hat_loss += c_hat_loss

        progress.set_description("loss: {},x_hat : {}, concat_hat : {}, z_hat : {}, c_hat : {}".format(batch_loss, batch_x_hat_loss, batch_concat_loss, batch_z_hat_loss, batch_c_hat_loss))

    return x_loss
    

In [22]:
Nonscale_imputataion = eval_model_ver2(model_ver4_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver4_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

Original MAE : tensor(14.5706, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2048, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.5876, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1131, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 61.01043701171875
------------------------------------------------------
Scale MAE : tensor(0.1665)
Scale MRE : tensor(0.2403)
Original MAE : 13.420484249847538
Original MRE : 0.18851546329660884
Train MAE : 8.96027212973667
Train MRE : 0.1056951522201209


In [20]:
# gamma h 위치 변경
model_ver4_ver2 = MGRU4_ver3(36, 64)
model_ver4_ver2.to(device)
loss_list = train_MGRU_ver2(model_ver4_ver2, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver4_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver4_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.65877914428711,x_hat : 20.156261444091797, concat_hat : 1.9235529899597168, z_hat : 22.005836486816406, c_hat : 17.57312774658203: 100%|██████████| 1000/1000 [12:55<00:00,  1.29it/s] 


ValueError: too many values to unpack (expected 6)

In [23]:
# gamma h 위치 변경
model_ver4_ver3 = MGRU4_ver3(36, 64)
model_ver4_ver3.to(device)
loss_list = train_MGRU_ver2(model_ver4_ver3, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver4_ver3, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver4_ver3, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.656925201416016,x_hat : 20.24460792541504, concat_hat : 1.8041247129440308, z_hat : 22.006738662719727, c_hat : 17.60144805908203: 100%|██████████| 1000/1000 [13:48<00:00,  1.21it/s] 


Original MAE : tensor(14.6565, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2060, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.6506, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1139, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 61.309410095214844
------------------------------------------------------
Scale MAE : tensor(0.1606)
Scale MRE : tensor(0.2319)
Original MAE : 12.974540261843785
Original MRE : 0.18225135717809676
Train MAE : 9.032765382444978
Train MRE : 0.10655028086677303


In [24]:
# gamma h 위치 변경
model_ver4_ver4 = MGRU4_ver3(36, 64)
model_ver4_ver4.to(device)
loss_list = train_MGRU_ver2(model_ver4_ver4, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver4_ver4, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver4_ver4, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.811378479003906,x_hat : 20.287670135498047, concat_hat : 1.8739075660705566, z_hat : 22.020198822021484, c_hat : 17.629608154296875: 100%|██████████| 1000/1000 [13:04<00:00,  1.28it/s]


Original MAE : tensor(14.2315, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2001, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.6205, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1135, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 61.15523910522461
------------------------------------------------------
Scale MAE : tensor(0.1635)
Scale MRE : tensor(0.2360)
Original MAE : 13.134113771522559
Original MRE : 0.18449286154909603
Train MAE : 8.991113220223388
Train MRE : 0.10605895297376208


In [25]:
# gamma h 위치 변경
model_ver4_ver4 = MGRU4_ver3(36, 64)
model_ver4_ver4.to(device)
loss_list = train_MGRU_ver2(model_ver4_ver4, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver4_ver4, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver4_ver4, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.68006896972656,x_hat : 20.250219345092773, concat_hat : 1.8196091651916504, z_hat : 22.009550094604492, c_hat : 17.600677490234375: 100%|██████████| 1000/1000 [11:18<00:00,  1.47it/s]


Original MAE : tensor(14.5136, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2040, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.6540, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1139, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 61.55867385864258
------------------------------------------------------
Scale MAE : tensor(0.1641)
Scale MRE : tensor(0.2369)
Original MAE : 13.21378483484492
Original MRE : 0.18561198863377773
Train MAE : 9.02566704426449
Train MRE : 0.10646654904215501


In [33]:
# gamma h 위치 변경
model_ver4_ver5 = MGRU4_ver3(36, 64)
model_ver4_ver5.to(device)
loss_list = train_MGRU_ver2(model_ver4_ver5, 0.001, 1500, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver4_ver5, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver4_ver5, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 60.141937255859375,x_hat : 19.597997665405273, concat_hat : 1.4211766719818115, z_hat : 21.97301483154297, c_hat : 17.149738311767578: 100%|██████████| 1500/1500 [15:38<00:00,  1.60it/s] 


Original MAE : tensor(14.3225, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2013, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.3857, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1107, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 59.691898345947266
------------------------------------------------------
Scale MAE : tensor(0.1632)
Scale MRE : tensor(0.2356)
Original MAE : 13.164091258085062
Original MRE : 0.18491395065904095
Train MAE : 8.757751583539656
Train MRE : 0.10330622478041106


In [28]:
class MGRU4_ver4(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU4_ver4, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 3, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        x_hat_loss = 0.0
        concat_loss = 0.0
        z_hat_loss = 0.0 
        c_hat_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            gamma_r = self.temp_decay_r(d)
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_hat_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_c = m * v + (1 - m) * x_hat

            RG = torch.cat([x_c, r], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            concat_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            a_c = m * v + (1 - m) * concat_hat

            z_hat = self.z_layer(a_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)
            z_hat_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_r, gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)
            c_hat_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid = hid * gamma_h
            
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_loss

In [30]:
# gamma r 추가
model_ver41_ver1 = MGRU4_ver4(36, 64)
model_ver41_ver1.to(device)
loss_list = train_MGRU_ver2(model_ver41_ver1, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.45452117919922,x_hat : 20.107765197753906, concat_hat : 1.9216434955596924, z_hat : 21.984661102294922, c_hat : 17.440452575683594: 100%|██████████| 1000/1000 [11:18<00:00,  1.47it/s]


Original MAE : tensor(14.5170, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2041, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.5544, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1127, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 61.028106689453125
------------------------------------------------------
Scale MAE : tensor(0.1622)
Scale MRE : tensor(0.2342)
Original MAE : 13.083672822268543
Original MRE : 0.18378432534870895
Train MAE : 8.926223073459308
Train MRE : 0.10529351037999823


In [31]:
# gamma r 추가
model_ver41_ver1 = MGRU4_ver4(36, 64)
model_ver41_ver1.to(device)
loss_list = train_MGRU_ver2(model_ver41_ver1, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.64366149902344,x_hat : 20.252798080444336, concat_hat : 1.8643040657043457, z_hat : 22.001829147338867, c_hat : 17.52472686767578: 100%|██████████| 1000/1000 [11:33<00:00,  1.44it/s] 


Original MAE : tensor(14.6448, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2059, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.5801, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1130, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 61.02008819580078
------------------------------------------------------
Scale MAE : tensor(0.1647)
Scale MRE : tensor(0.2378)
Original MAE : 13.272733256131211
Original MRE : 0.1864400279759136
Train MAE : 8.958068395436298
Train MRE : 0.10566915702388602


In [32]:
# gamma r 추가
model_ver41_ver1 = MGRU4_ver4(36, 64)
model_ver41_ver1.to(device)
loss_list = train_MGRU_ver2(model_ver41_ver1, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.37558364868164,x_hat : 20.133331298828125, concat_hat : 1.7785966396331787, z_hat : 21.99666976928711, c_hat : 17.46698570251465: 100%|██████████| 1000/1000 [11:20<00:00,  1.47it/s]  


Original MAE : tensor(14.8022, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2081, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.5391, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1125, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 60.74794006347656
------------------------------------------------------
Scale MAE : tensor(0.1634)
Scale MRE : tensor(0.2359)
Original MAE : 13.155537222774639
Original MRE : 0.18479379345013708
Train MAE : 8.924322593204288
Train MRE : 0.10527109236111028


In [37]:
# gamma r 추가
model_ver41_ver1 = MGRU4_ver4(36, 64)
model_ver41_ver1.to(device)
loss_list = train_MGRU_ver2(model_ver41_ver1, 0.001, 1000, dataset3, device)
Nonscale_imputataion = eval_model_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.38914108276367,x_hat : 20.12786865234375, concat_hat : 1.7794263362884521, z_hat : 22.015457153320312, c_hat : 17.466398239135742: 100%|██████████| 1000/1000 [11:20<00:00,  1.47it/s] 


Original MAE : tensor(14.7521, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Original MRE : tensor(0.2074, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)

Train MAE : tensor(9.5260, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
Train MRE : tensor(0.1124, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>)
--------------------------------------------------------------
x_loss: 60.77653503417969
------------------------------------------------------
Scale MAE : tensor(0.1648)
Scale MRE : tensor(0.2379)
Original MAE : 13.288042747698954
Original MRE : 0.18665507803237946
Train MAE : 8.907748658034722
Train MRE : 0.10507558662475881


In [38]:
class MGRU4_ver5(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU4_ver5, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        x_hat_loss = 0.0
        concat_loss = 0.0
        z_hat_loss = 0.0 
        c_hat_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            #gamma_r = self.temp_decay_r(d)
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_hat_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)
            x_c = m * v + (1 - m) * x_hat

            RG = torch.cat([x_c, r], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            concat_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)
            a_c = m * v + (1 - m) * concat_hat

            z_hat = self.z_layer(a_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)
            z_hat_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)
            c_hat_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid = hid * gamma_h
            
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_loss

In [43]:
# gamma r 제거
model_ver41_ver1 = MGRU4_ver5(36, 64)
model_ver41_ver1.to(device)
loss_list = train_MGRU_ver2(model_ver41_ver1, 0.001, 1000, dataset3, device)
#Nonscale_imputataion = eval_model_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.804649353027344,x_hat : 20.291366577148438, concat_hat : 1.8941487073898315, z_hat : 22.014728546142578, c_hat : 17.604400634765625: 100%|██████████| 1000/1000 [10:15<00:00,  1.62it/s]


------------------------------------------------------
Scale MAE : tensor(0.1615)
Scale MRE : tensor(0.2331)
Original MAE : 13.014369583839272
Original MRE : 0.18281083349422556
Train MAE : 9.001212083942123
Train MRE : 0.10617807892468713


In [42]:
# gamma r 제거
model_ver41_ver1 = MGRU4_ver5(36, 64)
model_ver41_ver1.to(device)
loss_list = train_MGRU_ver2(model_ver41_ver1, 0.001, 1000, dataset3, device)
#Nonscale_imputataion = eval_model_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver41_ver1, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.32242965698242,x_hat : 20.136425018310547, concat_hat : 1.6661280393600464, z_hat : 21.996843338012695, c_hat : 17.52303695678711: 100%|██████████| 1000/1000 [10:37<00:00,  1.57it/s] 


------------------------------------------------------
Scale MAE : tensor(0.1643)
Scale MRE : tensor(0.2372)
Original MAE : 13.253973369800402
Original MRE : 0.18617651075871006
Train MAE : 8.998195625051805
Train MRE : 0.10614249684894637


In [44]:
# gamma r 제거
model_ver41_ver2 = MGRU4_ver5(36, 64)
model_ver41_ver2.to(device)
loss_list = train_MGRU_ver2(model_ver41_ver2, 0.001, 1000, dataset3, device)
#Nonscale_imputataion = eval_model_ver2(model_ver41_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv", 64, 36)
print('------------------------------------------------------')
Nonscale_imputataion2 = eval_model2_ver2(model_ver41_ver2, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 61.70640563964844,x_hat : 20.27511978149414, concat_hat : 1.821865200996399, z_hat : 22.011383056640625, c_hat : 17.598037719726562: 100%|██████████| 1000/1000 [10:13<00:00,  1.63it/s]  


------------------------------------------------------
Scale MAE : tensor(0.1696)
Scale MRE : tensor(0.2448)
Original MAE : 13.659861268761906
Original MRE : 0.19187795519951584
Train MAE : 9.0522374945806
Train MRE : 0.10677997342816203
